In [1]:
## KAGGLE ONLY
# from shutil import copyfile
# copyfile(src="../input/inputs/generate_dataloaders.py", dst="../working/generate_dataloaders.py")
# copyfile(src="../input/inputs/train_dataloader.p", dst="../working/train_dataloader.p")
# copyfile(src="../input/inputs/val_dataloader.p", dst="../working/val_dataloader.p")
# copyfile(src="../input/inputs/centroids_dataloader.p", dst="../working/ground_truth_dataloader.p")
# copyfile(src="../input/inputs/dictionary.p", dst="../working/dictionary.p")

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

#from datasets import get_mnist_dataset, get_data_loader
#from utils import *
#from models import *

import pickle as pkl
import os
import datetime as dt
import pandas as pd
import random

from generate_dataloaders import *
from tqdm import tqdm_notebook as tqdm

import evaluation
import importlib
importlib.reload(evaluation)

<module 'evaluation' from '/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/evaluation.py'>

## Get Dataloaders

In [2]:
seed = 1029
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def _init_fn(worker_id):
    np.random.seed(int(seed))

In [3]:
path = os.getcwd()
data_dir = path + '/'
data_dir = path +'/data/' #Uncomment for local system

#### *Verify filenames are consistent*

In [4]:
train_loader = pkl.load(open(data_dir + 'train_dataloader.p','rb'))
train_loader_labelled = pkl.load(open(data_dir + 'train_labeled_dataloader.p','rb'))
train_loader_unlabelled = pkl.load(open(data_dir + 'train_unlabeled_dataloader.p','rb'))
val_loader = pkl.load(open(data_dir + 'val_dataloader.p','rb'))

In [5]:
review_dict = pkl.load(open(data_dir + 'dictionary.p','rb'))

In [6]:
#%conda install pytorch torchvision -c pytorch
## if torch.__version__ is not 1.3.1, run this cell then restart kernel

In [7]:
print(torch.__version__)

1.3.1


## PRE TRAINED WORD EMBEDDINGS 

In [8]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float16')

In [9]:
def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

In [10]:
def build_matrix(review_dict, embedding_index ,dim = 200):
#     embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(review_dict.tokens), dim))
    unknown_words = []
    
    for word, i in review_dict.ids.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

In [11]:
#glove_twitter = '../input/glove-global-vectors-for-word-representation/glove.twitter.27B.200d.txt' #Change loc for local system
glove_twitter = data_dir + 'glove.twitter.27B.200d.txt'

In [12]:
embedding_index = load_embeddings(glove_twitter)

In [13]:
glove_embedding_index,unknown_words = build_matrix(review_dict, embedding_index)
del embedding_index

In [14]:
len(review_dict.tokens)

16256

In [15]:
len(unknown_words)

4428

In [16]:
# for word in unknown_words:
#     print(word)

## Neural Network Class

NOTE: Data loader is defined as:
- tuple: (tokens, flagged_index, problematic)

In [16]:
def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False
        
def unfreeze_model(model):
    for param in model.parameters():
        param.requires_grad = True

In [17]:
class neuralNetBow_glove(nn.Module):
    """
    BagOfWords classification model
    """
    # NOTE: we can't use linear layer until we take weighted average, otherwise it will
    # remember certain positions incorrectly (ie, 4th word has bigger weights vs 7th word)
    def __init__(self, embedding_matrix, upweight=10):
        super(neuralNetBow_glove, self).__init__()
        self.embedding_matrix = embedding_matrix
        self.vocab_size = embedding_matrix.shape[0]
        self.embed_size = embedding_matrix.shape[1]
        
        self.embed = nn.Embedding(self.vocab_size, self.embed_size, padding_idx=0)
        self.embed.weight = nn.Parameter(torch.tensor(embedding_matrix,dtype=torch.float32))
        #self.embed.weight.requires_grad = unfrozen
        
        self.upweight = upweight
    
    def forward(self, tokens, flagged_index):
        batch_size, num_tokens = tokens.shape
        embedding = self.embed(tokens)
        
        # upweight by flagged_index
        embedding[torch.LongTensor(range(batch_size)),flagged_index.type(torch.LongTensor),:] *= self.upweight
        
        # average across embeddings
        embedding_ave = embedding.sum(1) / (num_tokens + self.upweight - 1)
        
        return embedding_ave

### Clustering Stuff

In [18]:
class KMeansCriterion(nn.Module):
    
    def __init__(self, lmbda):
        super().__init__()
        self.lmbda = lmbda
    
    def forward(self, embeddings, centroids):
        distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
        cluster_distances, cluster_assignments = distances.min(1)
        loss = self.lmbda * cluster_distances.sum()
        return loss, cluster_assignments

In [19]:
def centroid_init(k, d, dataloader, model, current_device):
    ## Here we ideally don't want to do randomized/zero initialization
    centroid_sums = torch.zeros(k, d).to(current_device)
    centroid_counts = torch.zeros(k).to(current_device)
    for (tokens, labels, flagged_indices) in dataloader:
        # cluster_assignments = torch.LongTensor(tokens.size(0)).random_(k)
        cluster_assignments = labels.to(current_device)
        
        model.eval()
        sentence_embed = model(tokens.to(current_device),flagged_indices.to(current_device))
    
        update_clusters(centroid_sums, centroid_counts,
                        cluster_assignments, sentence_embed.to(current_device))
    
    centroid_means = centroid_sums / centroid_counts[:, None].to(current_device)
    return centroid_means.clone()

def update_clusters(centroid_sums, centroid_counts,
                    cluster_assignments, embeddings):
    k = centroid_sums.size(0)

    centroid_sums.index_add_(0, cluster_assignments, embeddings)
    bin_counts = torch.bincount(cluster_assignments,minlength=k).type(torch.FloatTensor).to(current_device)
    centroid_counts.add_(bin_counts)
    
    #np_cluster_assignments = cluster_assignments.to('cpu')
    #np_counts = np.bincount(np_cluster_assignments.data.numpy(), minlength=k)
    #centroid_counts.add_(torch.FloatTensor(np_counts))

## Training Function

In [32]:
def train_model(model, centroids, criterion, train_loader, valid_loader, num_frozen=10, num_unfrozen=0, path_to_save=None, print_every = 1000):

    train_losses=[]
    val_losses=[]
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    # freeze part
    freeze_model(model)
    
    optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)
    
    for epoch in range(num_frozen):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.train()
        k, d = centroids.size()
        centroid_sums = torch.zeros_like(centroids).to(current_device)
        centroid_counts = torch.zeros(k).to(current_device)
        total_epoch_loss = 0

        # run one epoch of gradient descent on autoencoders wrt centroids
        for i, (tokens, labels, flagged_indices) in tqdm(enumerate(train_loader)):
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)

            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids.detach())

            # run update step
            optimizer.zero_grad()
            #cluster_loss.backward()
            optimizer.step()
            
            #Add loss to the epoch loss
            total_epoch_loss += cluster_loss.data

            # store centroid sums and counts in memory for later centering
            update_clusters(centroid_sums, centroid_counts,
                            cluster_assignments, sentence_embed)

            if i % print_every == 0:
                losses = cluster_loss.data/len(tokens)
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= len(train_loader.dataset)
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # update centroids based on assignments from autoencoders
        centroids = centroid_sums / (centroid_counts[:, None] + 1).to(current_device)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)
            
            #Add loss to the validation loss
            total_validation_loss += cluster_loss.data

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        print()
        
        if path_to_save == None:
            pass
        else:
            opts = {"embedding_matrix":model.embedding_matrix} #change options depending on model inputs required
            torch.save(model.state_dict(), path_to_save+'model_dict.pt')
            torch.save(centroids, path_to_save+'centroids')
            torch.save(train_losses, path_to_save+'train_losses')
            torch.save(val_losses, path_to_save+'val_losses')
            torch.save(opts, path_to_save+'opts')
            
            
    # unfreeze part
    unfreeze_model(model)
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            
    optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)
    
    for epoch in range(num_unfrozen):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.train()
        k, d = centroids.size()
        centroid_sums = torch.zeros_like(centroids).to(current_device)
        centroid_counts = torch.zeros(k).to(current_device)
        total_epoch_loss = 0

        # run one epoch of gradient descent on autoencoders wrt centroids
        for i, (tokens, labels, flagged_indices) in tqdm(enumerate(train_loader)):
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)

            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids.detach())

            # run update step
            optimizer.zero_grad()
            cluster_loss.backward()
            optimizer.step()
            
            #Add loss to the epoch loss
            total_epoch_loss += cluster_loss.data

            # store centroid sums and counts in memory for later centering
            update_clusters(centroid_sums, centroid_counts,
                            cluster_assignments, sentence_embed)

            if i % print_every == 0:
                losses = cluster_loss.data/len(tokens)
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= len(train_loader.dataset)
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # update centroids based on assignments from autoencoders
        centroids = centroid_sums / (centroid_counts[:, None] + 1).to(current_device)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)
            
            #Add loss to the validation loss
            total_validation_loss += cluster_loss.data

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            opts = {"embedding_matrix":model.embedding_matrix} #change options depending on model inputs required
            torch.save(model.state_dict(), path_to_save+'model_dict.pt')
            torch.save(centroids, path_to_save+'centroids')
            torch.save(train_losses, path_to_save+'train_losses')
            torch.save(val_losses, path_to_save+'val_losses')
            torch.save(opts, path_to_save+'opts')
        
    return model, centroids, train_losses, val_losses

In [34]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

model = neuralNetBow_glove(glove_embedding_index).to(current_device)

In [35]:
centroids = centroid_init(2, 200,train_loader_labelled, model, current_device)
criterion = KMeansCriterion(1).to(current_device)
#optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)

In [36]:
centroids

tensor([[ 7.3920e-02,  4.7916e-02, -3.4403e-03,  6.2405e-02, -1.4993e-02,
          9.7727e-02,  3.1345e-01, -2.0820e-02, -5.8466e-02, -6.3985e-02,
         -2.5678e-02, -6.1849e-02, -4.1237e-01, -5.8385e-02, -2.4222e-02,
          4.8410e-02, -1.2295e-02,  2.9833e-02, -7.6221e-02, -2.1353e-02,
         -1.3972e-02,  7.9938e-03, -3.5308e-02,  1.9144e-02, -6.5173e-02,
          4.9148e-01,  2.8338e-03,  8.5547e-02,  1.0131e-01,  3.7258e-03,
         -2.4145e-02, -7.5183e-02, -4.1977e-02, -5.0204e-03,  3.2769e-02,
          5.0823e-02,  3.5903e-02, -2.0590e-02,  2.4465e-02,  1.4467e-02,
          2.2642e-01, -1.7756e-02,  7.1540e-02,  9.6466e-03,  1.8655e-02,
         -3.1804e-02,  9.3824e-02, -1.1422e-02, -6.5178e-02,  2.9992e-02,
          2.2362e-02,  3.6858e-02, -5.0271e-02, -1.5484e-02,  5.2415e-03,
          2.4776e-02, -3.1752e-02, -1.2649e-02, -2.4775e-02, -1.0467e-02,
          1.2805e-02,  3.1579e-02, -3.3153e-02, -2.3706e-02,  5.6861e-02,
          3.1290e-02, -3.6886e-02, -2.

In [37]:
current_device

'cpu'

In [38]:
review_dict.get_id("the")

41

In [39]:
torch.tensor([41])

tensor([41])

In [40]:
model.embed(torch.tensor([41]).to(current_device))

tensor([[ 4.9341e-01,  3.5693e-01,  6.6064e-01, -3.2990e-02,  2.4988e-01,
          2.5928e-01, -2.7176e-02,  6.8420e-02, -2.9053e-01, -4.5703e-01,
         -7.7942e-02,  3.2520e-01, -1.4854e+00, -6.7444e-02, -1.7029e-01,
         -9.2926e-03,  3.4619e-01, -1.1574e-02,  3.7964e-02,  4.5605e-01,
          8.0505e-02,  1.5308e-01, -1.5308e-01, -1.8811e-01, -1.8201e-01,
          8.7256e-01,  3.9795e-01,  4.0991e-01,  4.4971e-01, -1.9646e-03,
         -4.1138e-02, -4.7882e-02, -2.4048e-01, -8.6853e-02,  1.4183e-02,
         -2.3755e-01,  2.5171e-01,  2.8540e-01,  4.4507e-01, -4.9634e-01,
         -1.2708e-01, -1.7480e-01,  8.2214e-02,  4.5410e-02,  5.1709e-01,
          3.4546e-02, -8.5815e-02, -3.4912e-01,  5.2197e-01, -3.9502e-01,
          6.4148e-02, -4.2017e-01, -1.5942e-01,  1.8286e-01, -5.7892e-02,
         -1.9180e-02, -4.4556e-01,  3.1543e-01, -1.6101e-01, -9.2163e-02,
         -2.4963e-01, -1.3895e-03, -4.2651e-01, -1.7932e-01,  8.1665e-02,
          1.8323e-01, -3.2056e-01, -1.

In [41]:
path = os.getcwd()
model_folder= 'baseline_unfrozen_glove/'
model_dir = path + '/models/' + model_folder

In [42]:
baseline_model, baseline_centroids, baseline_train_losses, baseline_val_losses = train_model(model, centroids, criterion, train_loader, val_loader, num_epochs=15, path_to_save=model_dir)


2019-11-25 15:04:42.798542 | Epoch 0


Average training loss at batch  0 : 1.755
Average training loss at batch  1000 : 2.087
Average training loss at batch  2000 : 1.899
Average training loss at batch  3000 : 1.660

Average training loss after epoch  0 : 1.985
Average validation loss after epoch  0 : 1.642
2019-11-25 15:04:48.816934 | Epoch 1


Average training loss at batch  0 : 1.410
Average training loss at batch  1000 : 1.543
Average training loss at batch  2000 : 1.704
Average training loss at batch  3000 : 2.023

Average training loss after epoch  1 : 1.649
Average validation loss after epoch  1 : 1.632
2019-11-25 15:04:55.299887 | Epoch 2


Average training loss at batch  0 : 1.722
Average training loss at batch  1000 : 1.108
Average training loss at batch  2000 : 1.784
Average training loss at batch  3000 : 1.714

Average training loss after epoch  2 : 1.636
Average validation loss after epoch  2 : 1.632
2019-11-25 15:05:05.823409 | Epoch 3


Average training loss at batch  0 : 1.391
Average training loss at batch  1000 : 1.524
Average training loss at batch  2000 : 1.521
Average training loss at batch  3000 : 1.333

Average training loss after epoch  3 : 1.635
Average validation loss after epoch  3 : 1.633
2019-11-25 15:05:12.508898 | Epoch 4


...

Average training loss at batch  0 : 1.485
Average training loss at batch  1000 : 0.120
Average training loss at batch  2000 : 0.140
Average training loss at batch  3000 : 0.084

Average training loss after epoch  0 : 0.181
Average validation loss after epoch  0 : 0.055
2019-11-25 15:09:43.109197 | Epoch 1


Average training loss at batch  0 : 0.102
Average training loss at batch  1000 : 0.039
Average training loss at batch  2000 : 0.023
Average training loss at batch  3000 : 0.022

Average training loss after epoch  1 : 0.035
Average validation loss after epoch  1 : 0.024
2019-11-25 15:16:22.268793 | Epoch 2


Average training loss at batch  0 : 0.024
Average training loss at batch  1000 : 0.012
Average training loss at batch  2000 : 0.011
Average training loss at batch  3000 : 0.023

Average training loss after epoch  2 : 0.019
Average validation loss after epoch  2 : 0.016
2019-11-25 15:22:30.719153 | Epoch 3


Average training loss at batch  0 : 0.010
Average training loss at batch  1000 : 0.010
Average training loss at batch  2000 : 0.016
Average training loss at batch  3000 : 0.012

Average training loss after epoch  3 : 0.014
Average validation loss after epoch  3 : 0.012
2019-11-25 15:27:54.740106 | Epoch 4


Average training loss at batch  0 : 0.012
Average training loss at batch  1000 : 0.012
Average training loss at batch  2000 : 0.014
Average training loss at batch  3000 : 0.006

Average training loss after epoch  4 : 0.010
Average validation loss after epoch  4 : 0.008


In [40]:
# #Only needed for Kaggle

# from IPython.display import FileLink, FileLinks 
# FileLinks('.') #lists all downloadable files on server

## Evaluate Model

To run this by itself, run:
- Get Dataloaders
- class definitions (model, clustering)

In [43]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

In [44]:
## This cell will change for each model
model_folder = 'baseline_unfrozen_glove/'

criterion = KMeansCriterion(1)
criterion = criterion.to(current_device)

path = os.getcwd()
model_dir = path + '/models/' + model_folder

opts = torch.load(model_dir+'opts')
model = neuralNetBow_glove(opts['embedding_matrix']) #change here depending on model
model.load_state_dict(torch.load(model_dir+'model_dict.pt',map_location=lambda storage, loc: storage))
model = model.to(current_device)
centroids = torch.load(model_dir+'centroids',map_location=lambda storage, loc: storage)

In [47]:
TP_cluster, FP_cluster=evaluation.main(model, centroids, val_loader, criterion, data_dir, current_device)

Total examples in val loader: 454
Assigned to cluster 1: 408
TP_rate: 0.9130434782608695
FP_rate: 0.08695652173913043
FN_rate: 0.8627450980392157
TN_rate: 0.13725490196078433


Accuracy: 0.525149190110827
Precision: 0.9130434782608695
Recall: 0.514162265962554
F1 score: 0.6578624078624078


/home/jubly/ENTER/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
